# Module3: Azure Machine Learning の基本操作トレーニング

Module2 で作成したモデルをワークスペースに登録し、Azure Container Instance（ACI）で Web サービスとしてデプロイします。

このノートブックでは、次のことを行います。

- Azure Machine Learning Workspace を作成します。
- Azure Machine Learning Workspace にモデルを登録する方法を学びます。
- モデルを AzureContainer インスタンスの Web サービスとしてデプロイします。


## 前提条件
Azure Machine Learning Notebook VM を使用している場合は、これで準備は完了です。それ以外の場合は、構成ノートブックを確認して Azure Machine Learning Python SDK をインストールし、ワークスペースを作成してください。

In [ ]:
import os
import json
import sklearn
import requests

import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model,InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException
import warnings
#Warning を抑止
warnings.simplefilter('ignore')
# azureml SDKのバージョンを確認する
print('azureml.core.VERSION: ', azureml.core.VERSION)

# 1. Azure Machine Learning ワークスペース

ワークスペースは、Azure Machine Learning の最上位のリソースで、Azure Machine Learning を使用するときに作成するすべての成果物を操作するための一元的な場所を提供します。 ワークスペースには、スクリプトのログ、メトリック、出力、スナップショットなど、すべてのトレーニング実行の履歴が保持されます。 この情報を使用して、最適なモデルを生成するトレーニング実行を判断します。

![image.png](https://docs.microsoft.com/ja-jp/azure/machine-learning/media/concept-workspace/azure-machine-learning-taxonomy.png)

この図は、ワークスペースの次のコンポーネントを示しています。
- ワークスペースには、Azure Machine Learning の実行に必要な Python 環境で構成されたクラウド リソースである Azure Machine Learning コンピューティング インスタンスを含めることができます。
- ユーザー ロールを使用すると、お使いのワークスペースを他のユーザー、チーム、またはプロジェクトと共有できます。
- コンピューティング ターゲットは、ご自身の実験の実行に使用されます。
- ワークスペースを作成すると、関連するリソースも自動的に作成されます。
- 実験は、ご自身のモデルの構築に使用するトレーニング実行です。
- パイプラインは、お使いのモデルをトレーニングおよび再トレーニングするための再利用可能なワークフローです。
- データセットは、モデルのトレーニングとパイプラインの作成に使用するデータの管理に役立ちます。
- デプロイするモデルを作成したら、登録済みモデルを作成します。
- 登録したモデルとスコアリング スクリプトを使用して、デプロイ エンドポイントを作成します。

## ワークスペースのセットアップ
まず、Azureサブスクリプションに関する次の情報を入力する必要があります。

**独自のAzureサブスクリプションを使用している場合は、使用するsubscription_id、resource_group、workspace_name、workspace_regionの名前を指定してください。** ワークスペースのタイプは [Machine Learning Workspace](https://docs.microsoft.com/azure/machine-learning/service/setup-create-workspace) である必要があることに注意してください。

**環境が提供されている場合は、以下の値のXXXXXを一意の識別子に置き換えてください。**

次のセルで、コメントの指示に従って、 `subscription_id`、` resource_group`、 `workspace_name`、および` workspace_region`の値を必ず設定してください (*これらの値は、Azureポータルから取得できます*).

これらの値を取得するには、次の操作を行います:

1. Azureポータルに移動し、提供された資格情報でログインします。

2. 左側のメニューの[お気に入り]で、[リソースグループ]を選択します。

3. リストで、「XXXXX」のような名前のリソースグループを選択します。

4. 「概要」タブから、必要な値を取得します。

上のコマンドバーの `>|Run` ボタンを選択して、次のセルを実行します。

In [ ]:
#既存のAzureサブスクリプションのサブスクリプションIDに置き換える
subscription_id = ""

#Azure ML関連のサービスを含むリソースグループの名前を指定します
resource_group = "amlailab"

#作成されるAzure Machine Learningワークスペースの一意の名前とリージョンを指定します
workspace_name = "amlailab-csml"
workspace_region = "eastus" # japaneast, eastus2, westus, westcentralus, southeastasia, australiaeast, westeurope

## ワークスペースを作成して接続する

Azure Machine Learning Python SDK は、Azure Machine Learning サービスの実験、モデル管理、モデルデプロイメント機能を活用するために必要です。次のセルを実行して、新しい Azure Machine Learning **Workspace** を作成し、構成をディスクに保存します。 `config.json` という名前の設定ファイルは、` .azureml` という名前のフォルダーに保存されます。

**重要**: セルの下に出力されるテキストでログインするように求められます。 表示されたURLに移動し、提供されているコードを入力してください。コードを入力したら、このノートブックに戻り、 `Workspace configuration succeeded` と出力されるのを待ちます。

**任意**: 複数のテナントを持っている場合、以下のセルをアクティブにしてください。クラスをインポートし、ターゲットとするテナントを明示的に定義する必要があります。 
InteractiveLoginAuthentication のコンストラクターを呼び出すと、指定したテナントIDに基づいてログインするよう求められます。

In [ ]:
#任意: 複数のテナントを持っている場合
#from azureml.core.authentication import InteractiveLoginAuthentication
#auth = InteractiveLoginAuthentication(tenant_id = '<your-tenant-id>')

上記セルをアクティブにした場合、下記の `Workspace.create` メソッドで `auth = auth` パラメータを有効化します。

In [ ]:
# 既存のワークスペースが存在する場合は既存の Workspace オブジェクトを返します。
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region, 
    exist_ok = True
#   ,auth = auth
    )

print(ws)

複数の環境で同じワークスペースを使用するには、JSON 構成ファイルを作成します。 構成ファイルを使用すると、サブスクリプション、リソース、およびワークスペース名が保存されるため、簡単に読み込むことができます。 構成を保存するには、write_config メソッドを使用します。

In [ ]:
ws.write_config()
print('Workspace configuration succeeded')

Workspace.from_config() メソッドを使用すると、複数の Python ノートブックまたはプロジェクト間で同じワークスペースを再利用することが簡単にできます。 ユーザーは、この機能を使用してワークスペース ARM プロパティを保存し、from_config を使用して、ワークスペースの ARM プロパティを再入力することなく、さまざまな Python ノートブックまたはプロジェクトで同じワークスペースを読み込むことができます。

In [ ]:
ws = Workspace.from_config()
print(ws.get_details())

# 2. Azure Machine Learning にモデルを登録する

以下では、モデルをワークスペースに登録します（これにより、コピーがクラウドに保存されます）。
モデルにタグと説明を追加できます。現在のディレクトリにある BostonPrediction.pkl ファイルを、ワークスペースで BostonPrediction_model という名前のモデルとして使用しています。

In [ ]:
model = Model.register(model_path="BostonPrediction.pkl",
                       model_name="BostonPrediction_model",
                       tags={'area': "rm", 'type': "regression"},
                       description="Linear regression model to predict rm",
                       workspace=ws)

print(model.name, model.description, model.version)

モデルが正常に登録されたら、Azure Machine Learning Studio の**モデル**メニューから参照・管理することができます。

## 既存モデルをロードする

Azure Machine Learning にモデルを登録したら、以降は以下のコードを使ってモデル名を指定してロードすることができます。

In [ ]:
model = Model(ws, name='BostonPrediction_model')

print(model.name, model.description, model.version)

# 3. Azure Container Instance（ACI）にモデルをデプロイする

## スコアリングWebサービスを作成する

Azure Machine Learning サービスでスコアリング用のモデルをデプロイする場合、モデルをロードしてスコアリングに使用する単純な Web サービスのコードを定義する必要があります。慣例により、このサービスには、モデルをロードする init と、ロードされたモデルを使用してデータをスコアリングする run の2つのメソッドがあります。

このスコアリングサービスコード(score.py)は、後で特別に準備された Docker コンテナー内にデプロイされます。

In [ ]:
#スコアリングサービスコード(score.py)の保存先
source_directory = "source_directory"
os.makedirs(source_directory, exist_ok=True)

In [ ]:
%%writefile source_directory/score.py
# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------
import json
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from azureml.core import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

input_sample = pd.DataFrame({"crim": pd.Series(["0.02985"], dtype="float64"), "zn": pd.Series(["0.0"], dtype="float64"), "indus": pd.Series(["2.18"], dtype="float64"), "chas": pd.Series(["0.0"], dtype="float64"), "nox": pd.Series(["0.458"], dtype="float64"), "rm": pd.Series(["6.43"], dtype="float64"), "age": pd.Series(["58.7"], dtype="float64"), "dis": pd.Series(["6.0622"], dtype="float64"), "rad": pd.Series(["3.0"], dtype="float64"), "tax": pd.Series(["222.0"], dtype="float64"), "ptratio": pd.Series(["18.7"], dtype="float64"), "black": pd.Series(["394.12"], dtype="float64"), "lstat": pd.Series(["5.21"], dtype="float64")})
output_sample = np.array([0])

def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = Model.get_model_path('BostonPrediction_model')
    try:
        model = joblib.load(model_path)
        print("model:", model, type(model))
    except Exception as e:
        error = str(e)
        return error

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        #data:   <class 'pandas.core.frame.DataFrame'>
        #result: <class 'numpy.ndarray'>
        print("data", data, type(data))
        result = model.predict(data)
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

## 環境(Environment)

Azure ML Environment は、機械学習トレーニングが行われる環境をカプセル化したものです。これらは、Python パッケージ、環境変数、Docker 設定、およびその他の属性を宣言型で定義します。環境はバージョン管理されています。環境を更新し、古いバージョンを取得して、作業を再検討および確認できます。

環境により、次のことが可能になります:

- Python パッケージやそのバージョンなど、トレーニングプロセスの依存関係をカプセル化します
- VM または MLCompute クラスターでのリモート実行でローカルコンピューター上の Python 環境を再現します
- 実稼働環境で実験環境を再現します
- 既存のモデルがトレーニングされた環境を再検討して監査します

環境、計算ターゲット、トレーニングスクリプトが一緒になって、実行構成を形成します。

In [ ]:
myenv = Environment('myenv')
myenv.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
myenv.python.conda_dependencies.add_pip_package("joblib")
myenv.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

# explicitly set base_image to None when setting base_dockerfile
myenv.docker.base_image = None
myenv.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04\nRUN echo \"this is test\""
myenv.inferencing_stack_version = "latest"


## デプロイ

モデルの実行方法をより制御したい場合、別のフレームワークを使用している場合、または特別なランタイム要件がある場合は、独自の環境とスコアリング方法を指定することができます。カスタム環境は、デプロイしたいモデルに使用することができます。

前のコードでは、Environment オブジェクトを作成し、モデルに必要な CondaDependencies を提供することで、モデルの実行環境を指定しました。

次のセルでは、Azure Machine Learning SDK を使用して、モデルとスコアリングスクリプトをコンテナにパッケージ化し、そのコンテナを Azure コンテナインスタンスにデプロイします。

以下のセルを実行します。

In [ ]:
inference_config = InferenceConfig(source_directory=source_directory,
                                   entry_script="score.py",
                                   environment=myenv)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Prediction'}, 
    description = 'Linear regression model to predict rm')

これで、Azure コンテナインスタンスへのデプロイを開始する準備が整いました。

次のセルを実行してください: _Running_ タグで進行状況のドットが追加されている間、完了まで `5～15分` ほどかかるかもしれません。

Web サービスの準備ができたら、以下のような出力が表示されます。

```
Succeeded
ACI service creation operation finished, operation "Succeeded"
```

In [ ]:
# Azure Machine Learning のエンドポイントに登録される名前
service_name = "bostonprediction"

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass


webservice = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

webservice.wait_for_deployment(show_output=True, timeout_sec=1800)

# デプロイしたWebサービスを削除する
#Webservice(ws, "bostonprediction").delete()

デプロイが完了したら、Azure Machine Learning Studio のアセット->エンドポイント->リアルタイムエンドポイントにサービスが登録されていることが確認できます。

## デプロイされたサービスをテストする

これで、デプロイされた Web サービスを使用してスコアリングをテストする準備ができました。 次のセルは Web サービスを呼び出します。

ここでは、Webservice クラスの run メソッドを利用して、推論リクエストを POST します。インプットデータは JSON 文字列に整形して送信する必要があります

In [ ]:
#引数の型
#pd.DataFrame({"crim": pd.Series(["0.02985"], dtype="float64"), "zn": pd.Series(["0.0"], dtype="float64"), "indus": pd.Series(["2.18"], dtype="float64"), "chas": pd.Series(["0.0"], dtype="float64"), "nox": pd.Series(["0.458"], dtype="float64"), "rm": pd.Series(["6.43"], dtype="float64"), "age": pd.Series(["58.7"], dtype="float64"), "dis": pd.Series(["6.0622"], dtype="float64"), "rad": pd.Series(["3.0"], dtype="float64"), "tax": pd.Series(["222.0"], dtype="float64"), "ptratio": pd.Series(["18.7"], dtype="float64"), "black": pd.Series(["394.12"], dtype="float64"), "lstat": pd.Series(["5.21"], dtype="float64")})

#JSON文字列に整形する
input_payload = json.dumps({
    'data': [
        [0, 0.0, 2.18, 0.0, 0.458, 5.43, 58.7, 6.0622, 3.0, 222.0, 18.7, 394.12, 0]
    ]
})
#リクエストをPOST
output = webservice.run(input_payload)
print(output)

Docker サービスのログを取得する。

In [ ]:
print(webservice.get_logs())

## スコアリングURIをキャプチャする

RESTクライアントからサービスを呼び出すには、スコアリングURIを取得する必要があります。 次のセルを実行してスコアリングURIを取得し、この値をメモします。最後のノートブックで必要になります。

In [ ]:
scoring_uri = webservice.scoring_uri
print(scoring_uri)

このサービスのデプロイに使用されるデフォルト設定では、認証を必要としないサービスが作成されるため、このサービスを呼び出すために必要な値はスコアリングURIのみです。

## スコアリングURIにHTTP呼び出しを行う

次は、取得したスコアリングURIに対して、Webservice.run() を使わずに、POST リクエストを送信してみましょう。単に requests ライブラリの post メソッドを使用するだけです。

In [ ]:
#scoring_uri
ml_service_scoring_endpoint = scoring_uri

input_payload = json.dumps({
    'data': [
        [ 0,0.0,2.18,0.0,0.458,5.43,58.7,6.0622,3.0,222.0,18.7,394.12, 0]
    ]
})

headers = {
    "Content-Type" : "application/json",
    "Authorization": "Bearer "
}

response  = requests.post(ml_service_scoring_endpoint, headers=headers, data=input_payload)
result = response.json()
print(result)